In [1]:
"""Visualization utilies."""

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


def show_points(points, title):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.set_title(title)
    ax.set_xlim3d([-2, 2])
    ax.set_ylim3d([-2, 2])
    ax.set_zlim3d([0, 4])
    ax.scatter(points[:, 0], points[:, 2], points[:, 1])

def compare_points(points1, points2):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.set_xlim3d([-2, 2])
    ax.set_ylim3d([-2, 2])
    ax.set_zlim3d([0, 4])
    ax.scatter(points1[:, 0], points1[:, 2], points1[:, 1])
    ax.scatter(points2[:, 0], points2[:, 2], points2[:, 1])

In [2]:
# Load the canonical-space object point clouds
import trimesh
import collada
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

def fps_downsample(points, number_of_points_to_sample):
  selected_points = np.zeros((number_of_points_to_sample, 3))
  dist = np.ones(points.shape[0]) * np.inf # distance to the selected set
  for i in tqdm(range(number_of_points_to_sample)):
      # pick the point with max dist
      idx = np.argmax(dist)
      selected_points[i] = points[idx]
      dist_ = ((points - selected_points[i]) ** 2).sum(-1)
      dist = np.minimum(dist, dist_)
  return selected_points

object_models_file = "models/objects_v1.csv"
object_models_info = pd.read_csv(object_models_file)
object_models_name = object_models_info["object"].to_list()
object_models_location = object_models_info["location"].to_list()
object_models_num = len(object_models_name)


object_models = {}
for object_name, object_mesh_dir in zip(object_models_name, object_models_location):
  object_mesh_path = os.path.join(object_mesh_dir, "visual_meshes", "visual.dae")
  # Collada-based loading
  object_mesh = trimesh.exchange.dae.load_collada(object_mesh_path)
  mesh_name = object_mesh['graph'][0]['geometry']
  object_models[object_name] = object_mesh['geometry'][mesh_name]['vertices']
  # Trimesh-based loading
  # object_mesh = trimesh.load(object_mesh_path, force='mesh')
  # object_models[object_name] = object_mesh.vertices
  # print(object_mesh)

# object_models with key=objectName, value=objectPointCloud
# print(object_models_info)
print(object_models["jenga"])
print(len(object_models["jenga"]))
# print(np.unique(object_models["jenga"]))
# print(len(object_models))







[[-0.07499998 -0.02499997  0.01499998]
 [ 0.07499998 -0.02499997  0.01499998]
 [ 0.07499998  0.02499997  0.01499998]
 [-0.07499998 -0.02499997  0.01499998]
 [ 0.07499998  0.02499997  0.01499998]
 [-0.07499998  0.02499997  0.01499998]
 [ 0.07499998  0.02499997 -0.01499998]
 [ 0.07499998 -0.02499997 -0.01499998]
 [-0.07499998 -0.02499997 -0.01499998]
 [-0.07499998  0.02499997 -0.01499998]
 [ 0.07499998  0.02499997 -0.01499998]
 [-0.07499998 -0.02499997 -0.01499998]
 [-0.07499998  0.02499997  0.01499998]
 [-0.07499998  0.02499997 -0.01499998]
 [-0.07499998 -0.02499997 -0.01499998]
 [-0.07499998 -0.02499997  0.01499998]
 [-0.07499998  0.02499997  0.01499998]
 [-0.07499998 -0.02499997 -0.01499998]
 [ 0.07499998 -0.02499997 -0.01499998]
 [ 0.07499998  0.02499997 -0.01499998]
 [ 0.07499998  0.02499997  0.01499998]
 [ 0.07499998 -0.02499997 -0.01499998]
 [ 0.07499998  0.02499997  0.01499998]
 [ 0.07499998 -0.02499997  0.01499998]
 [ 0.07499998 -0.02499997  0.01499998]
 [-0.07499998 -0.02499997

In [3]:
"""Data loading helpers"""
import numpy as np
import json
import matplotlib.pyplot as plt
import os
from PIL import Image
import pickle
import utils
import open3d as o3d
# # Helper function for the reconstruction of the target point cloud
# rgb = np.array(Image.open(rgb_files[0])) / 255   # convert 0-255 to 0-1
# depth = np.array(Image.open(depth_files[0])) / 1000   # convert from mm to m
# label = np.array(Image.open(label_files[0]))
# meta = load_pickle(meta_files[0])
# intrinsic = meta['intrinsic']
# z = depth
# v, u = np.indices(z.shape)
# uv1 = np.stack([u + 0.5, v + 0.5, np.ones_like(z)], axis=-1)
# points_viewer = uv1 @ np.linalg.inv(intrinsic).T * z[..., None]  # [H, W, 3]
# # print(points_viewer.shape)
# # print(points_viewer.shape[0]*points_viewer.shape[1])
# # print(points_viewer)

def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def dump_json(sample, path):
  with open(path, 'w') as fp:
    json.dump(sample, fp)
  return 0

def load_json(path):
  f = open(path)
  data = json.load(f)
  return data

def get_point_cloud(depth, intrinsic):
  z = depth
  v, u = np.indices(z.shape)
  uv1 = np.stack([u + 0.5, v + 0.5, np.ones_like(z)], axis=-1)
  points_viewer = uv1 @ np.linalg.inv(intrinsic).T * z[..., None]  # [H, W, 3]
  return points_viewer

def pts_to_o3d_pcd(pts):
  """Transform to o3d pcd"""
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(pts)
  return pcd

def o3dvis(pts):
    o3d.visualization.draw_geometries([pts_to_o3d_pcd(pt) for pt in pts])
    return 0

def get_object_point_cloud(test_image_label, object_id, test_depth, intrinsic):
  # print(np.where(test_image_label==object_id))
  # print(test_image_label[327][654])
  test_image_label[np.where(test_image_label==object_id)] = 255
  # print(test_image_label[327][654])
  # print(np.where(test_image_label==255))
  test_image_label[np.where(test_image_label!=255)] = 0
  test_image_label[np.where(test_image_label==255)] = 1
  test_object_depth = test_depth * test_image_label
  test_pcd_target = get_point_cloud(test_object_depth, intrinsic)
  # (H, W, dim) = test_pcd_target.shape
  # filter out target object point cloud
  # print(test_pcd_target.shape)
  # print(test_pcd_target)
  # print((test_pcd_target[:,0]!=0)|(test_pcd_target[:,1]!=0)|(test_pcd_target[:,2]!=0))
  test_pcd_target = test_pcd_target.reshape(-1, test_pcd_target.shape[-1]) # reshape to (H*W, 3)
  test_pcd_target = test_pcd_target[(test_pcd_target[:,0]!=0)|(test_pcd_target[:,1]!=0)|(test_pcd_target[:,2]!=0)]
  return test_pcd_target


def get_meta(meta_path):
  return load_pickle(meta_path)

def get_depth(depth_path):
  return (np.array(Image.open(depth_path))/1000)

def get_label(label_path):
  return np.array(Image.open(label_path))




"""Metric and visualization."""

def compute_rre(R_est: np.ndarray, R_gt: np.ndarray):
    """Compute the relative rotation error (geodesic distance of rotation)."""
    assert R_est.shape == (3, 3), 'R_est: expected shape (3, 3), received shape {}.'.format(R_est.shape)
    assert R_gt.shape == (3, 3), 'R_gt: expected shape (3, 3), received shape {}.'.format(R_gt.shape)
    # relative rotation error (RRE)
    # Rotational degree loss (not objective of optimization)
    rre = np.arccos(np.clip(0.5 * (np.trace(R_est.T @ R_gt) - 1), -1.0, 1.0))
    return rre


def compute_rte(t_est: np.ndarray, t_gt: np.ndarray):
    assert t_est.shape == (3,), 't_est: expected shape (3,), received shape {}.'.format(t_est.shape)
    assert t_gt.shape == (3,), 't_gt: expected shape (3,), received shape {}.'.format(t_gt.shape)
    # relative translation error (RTE)
    rte = np.linalg.norm(t_est - t_gt) # Resembling MSE loss
    return rte




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
"""Visualization on comparing between train pose-transformed point clouds."""
import open3d as o3d
import numpy as np
import time
import copy

# def draw_registration_result(source, target, transformation):
#     source_temp = copy.deepcopy(source)
#     target_temp = copy.deepcopy(target)
#     source_temp.paint_uniform_color([1, 0.706, 0])
#     target_temp.paint_uniform_color([0, 0.651, 0.929])
#     source_temp.transform(transformation)
#     o3d.visualization.draw_geometries([source_temp, target_temp],
#                                       zoom=0.4559,
#                                       front=[0.6452, -0.3036, -0.7011],
#                                       lookat=[1.9892, 2.0208, 1.8945],
#                                       up=[-0.2779, -0.9482, 0.1556])

# def get_o3d_icp_tensor(source_pcd, target_pcd):
#   # T = icp(source_pcd, target_pcd, 150)
#   source_pcd_o3d, target_pcd_o3d = o3d.t.geometry.PointCloud(), o3d.t.geometry.PointCloud()
#   # print(source_pcd)
#   source_pcd_o3d.point.positions = o3d.core.Tensor(source_pcd, dtype=o3d.core.Dtype.Float64)
#   # source_pcd_o3d.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(knn=50))
#   target_pcd_o3d.point.positions = o3d.core.Tensor(target_pcd, dtype=o3d.core.Dtype.Float64)
#   # Search distance for Nearest Neighbour Search [Hybrid-Search is used].
#   max_correspondence_distance = 1
#   init = o3d.core.Tensor.eye(4, o3d.core.Dtype.Float64)
#   # Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
#   treg = o3d.cpu.pybind.t.pipelines.registration
#   estimation = treg.TransformationEstimationPointToPoint()
#   callback_after_iteration = lambda updated_result_dict : print("Iteration Index: {}, Fitness: {}, Inlier RMSE: {},".format(
#     updated_result_dict["iteration_index"].item(), \
#     updated_result_dict["fitness"].item(), \
#     updated_result_dict["inlier_rmse"].item())) \

#   # Convergence-Criteria for Vanilla ICP
#   criteria = treg.ICPConvergenceCriteria(relative_fitness=0.0000000000001,
#                       relative_rmse=0.0000000000001,
#                       max_iteration=100)

#   # Down-sampling voxel-size.
#   # voxel_size = 0.0016625
#   # voxel_size = 0.003125
#   voxel_size = 0.00625
#   # voxel_size = 0.0125

#   # Save iteration wise `fitness`, `inlier_rmse`, etc. to analyse and tune result.
#   save_loss_log = True
#   s = time.time()

#   # voxel_size = 0.05  # means 5cm for this dataset
#   # source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, source_pcd_o3d, target_pcd_o3d)
#   # result_ransac = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
#   # init = o3d.core.Tensor(result_ransac.transformation, dtype=o3d.core.Dtype.Float64)

#   registration_icp = treg.icp(source_pcd_o3d, target_pcd_o3d, max_correspondence_distance, init, estimation, criteria, voxel_size, callback_after_iteration)
#   icp_time = time.time() - s
#   T = registration_icp.transformation
#   T = T.numpy()
#   return T

# def execute_fast_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
#   distance_threshold = voxel_size * 0.5
#   print(":: Apply fast global registration with distance threshold %.3f" \
#           % distance_threshold)
#   result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
#       source_down, target_down, source_fpfh, target_fpfh,
#       o3d.pipelines.registration.FastGlobalRegistrationOption(
#           maximum_correspondence_distance=distance_threshold))
#   return result

# def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size, init):
#     distance_threshold = voxel_size * 0.4
#     print(":: Point-to-plane ICP registration is applied on original point")
#     print("   clouds to refine the alignment. This time we use a strict")
#     print("   distance threshold %.3f." % distance_threshold)
#     result = o3d.pipelines.registration.registration_icp(
#         source, target, distance_threshold, init,
#         o3d.pipelines.registration.TransformationEstimationPointToPlane())
#     return result




















icp_global_voxel_size = 0.03
icp_dist_threshold = 5
init_random_time = 100
max_preproc_nn_normal = 50
max_preproc_nn_fpfh = 100
tgSrcFactor = 1.3
dist_voxel_factor = 3
fps_dnsamp_factor = 2000
# cdistFrac = 0
cdistFrac = 0.1
# heavy_dnsamp_frac = 9/14
heavy_dnsamp_frac = 1/2
# heavy_dnsamp_frac = 5/14
icp_relative_fitness = 0.00001
icp_relative_rmse = 0.00001
icp_max_iteration = 500
# light_heavy_dnsamp_vxsize_ratio = 0
light_heavy_dnsamp_vxsize_ratio = 0.001
ransac_max_iteration = 10000 
ransac_max_validation = 0.9999
# heavy_dnsamp_frac = 5/7
# heavy_dnsamp_frac = 4/7

# Define the data point to visualize
train_vis_varianct, train_vis_index, object_ids, vis_id = "1-1-15", 0, [14, 35, 48, 56, 58], 48
rgb_file = "datas/{}_color_kinect.png".format(train_vis_varianct)
depth_file = "datas/{}_depth_kinect.png".format(train_vis_varianct)
label_file = "datas/{}_label_kinect.png".format(train_vis_varianct)
meta_file = "datas/{}_meta.pkl".format(train_vis_varianct)

# object-based parameter adjustment
# if vis_id==48:
#   # icp_global_voxel_size = 0.05
#   tgSrcFactor = 1.5
#   # light_heavy_dnsamp_vxsize_ratio = 0.1
#   # heavy_dnsamp_frac = 4.5/7


def get_o3d_icp(source_pcd, target_pcd, init=np.eye(4), icp_dist_threshold=icp_dist_threshold,
                icp_relative_fitness=icp_relative_fitness,
                icp_relative_rmse=icp_relative_rmse,
                icp_max_iteration=icp_max_iteration
                ):
  # T = icp(source_pcd, target_pcd, 150)
  source_pcd_o3d, target_pcd_o3d = o3d.geometry.PointCloud(), o3d.geometry.PointCloud()
  # print(source_pcd)
  source_pcd_o3d.points = o3d.utility.Vector3dVector(source_pcd)
  # source_pcd_o3d.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(knn=50))
  target_pcd_o3d.points = o3d.utility.Vector3dVector(target_pcd)
  # target_pcd_o3d.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(knn=50))
  # print(source_pcd_o3d)
  # print(target_pcd_o3d)
  # init = np.random.rand(4,4)
  # init = -1 + init * 2
  # init[-1] = np.zeros(4)
  # print(init)
  # init[0,0] = 2
  T = o3d.pipelines.registration.registration_icp( \
        source_pcd_o3d, target_pcd_o3d, icp_dist_threshold, init, \
        o3d.pipelines.registration.TransformationEstimationPointToPoint(), \
        # o3d.pipelines.registration.TransformationEstimationPointToPlane(), \
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=icp_relative_fitness, \
                                 relative_rmse=icp_relative_rmse, \
                                 max_iteration=icp_max_iteration))
  # T = icp(source_pcd, target_pcd)
  # print(T.inlier_rmse)
  # print(T.fitness)
  # print(T.correspondence_set)
  return T

def half_voxel_dnsample(source, voxel_size, heavy_dnsamp_frac=heavy_dnsamp_frac, 
                        light_heavy_dnsamp_vxsize_ratio=light_heavy_dnsamp_vxsize_ratio):
  pts = source.points
  size = len(pts)
  # print(pts.shape)
  first_half = o3d_Vec_to_o3d_pcd(pts[:int(size*heavy_dnsamp_frac)])
  first_half = first_half.voxel_down_sample(voxel_size=voxel_size) # heavily down-sampled side
  secn_half = o3d_Vec_to_o3d_pcd(pts[int(size*heavy_dnsamp_frac):])
  if (light_heavy_dnsamp_vxsize_ratio != 0):
    secn_half = secn_half.voxel_down_sample(voxel_size=voxel_size*light_heavy_dnsamp_vxsize_ratio) # weakly down-sampled side

  first_half_pts = np.asarray(first_half.points) 
  secn_half_pts = np.asarray(secn_half.points)
  full_pcd_pts = np.vstack((first_half_pts, secn_half_pts))
  full_pcd_pts = pts_to_o3d_pcd(full_pcd_pts)
  return full_pcd_pts

def half_farthest_dnsample(source, fps_rates=[1/64, 1/32]):
  pts = source.points
  size = len(pts)
  # print(pts.shape)
  first_half = o3d_Vec_to_o3d_pcd(pts[:int(size*heavy_dnsamp_frac)])
  first_half = first_half.farthest_point_down_sample(int(size * fps_rates[0])) # heavily down-sampled side
  secn_half = o3d_Vec_to_o3d_pcd(pts[int(size*heavy_dnsamp_frac):])
  if (fps_rates[1] != 1):
    secn_half = secn_half.farthest_point_down_sample(int(size * fps_rates[1])) # weakly down-sampled side

  first_half_pts = np.asarray(first_half.points) 
  secn_half_pts = np.asarray(secn_half.points)
  full_pcd_pts = np.vstack((first_half_pts, secn_half_pts))
  full_pcd_pts = pts_to_o3d_pcd(full_pcd_pts)
  return full_pcd_pts

def preprocess_point_cloud(source, target, 
                           voxel_size, max_nn_normal=max_preproc_nn_normal, 
                           max_nn_fpfh=max_preproc_nn_fpfh, tgSrcFactor=tgSrcFactor, fps_dnsamp_factor=fps_dnsamp_factor):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    # pts = pcd.points
    # size = len(source.points)
    # if (len(target.points) > 1000):
    #   target = target.farthest_point_down_sample(len(target.points))
    size_factor = len(source.points) / len(target.points)
    if (size_factor >= 1):
      # print(len(target.points), fps_dnsamp_factor, len(target.points)//fps_dnsamp_factor)
      # src_pcd_down = source.farthest_point_down_sample(len(source.points)//fps_dnsamp_factor)
      # src_pcd_down = source.voxel_down_sample(voxel_size=0.019)
      # src_pcd_down = source.voxel_down_sample(voxel_size=voxel_size)
      src_pcd_down = half_voxel_dnsample(source, voxel_size=voxel_size)
      # src_pcd_down = half_farthest_dnsample(source)
    else: 
      src_pcd_down = source
    # src_pcd_down = source
    tg_pcd_down = target
    # tg_pcd_down = target.farthest_point_down_sample(int(tgSrcFactor*len(src_pcd_down.points)))
    # src_pcd_down = source.farthest_point_down_sample(int(size/5))
    # src_pcd_down = source


    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    src_pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=max_nn_normal))
    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    src_pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        src_pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn_fpfh))


    print(":: Estimate normal with search radius %.3f." % radius_normal)
    tg_pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=max_nn_normal))
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    tg_pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        tg_pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn_fpfh))

    return src_pcd_down, src_pcd_fpfh, tg_pcd_down, tg_pcd_fpfh

def prepare_dataset(voxel_size, source, target):
    print(":: Load two point clouds and disturb initial pose.")

    # trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
    #                          [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    # source.transform(trans_init)

    # source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    # target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    source_down, source_fpfh, target_down, target_fpfh = preprocess_point_cloud(source, target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size, dist_voxel_factor=dist_voxel_factor,
                                ransac_max_iteration=ransac_max_iteration,
                                ransac_max_validation=ransac_max_validation):
  distance_threshold = voxel_size * dist_voxel_factor
  # print(":: RANSAC registration on downsampled point clouds.")
  # print("   Since the downsampling voxel size is %.3f," % voxel_size)
  # print("   we use a liberal distance threshold %.3f." % distance_threshold)
  result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
      source_down, target_down, source_fpfh, target_fpfh, True,
      distance_threshold,
      o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
      3, [
          o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
              0.9),
          o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
              distance_threshold)
      ], o3d.pipelines.registration.RANSACConvergenceCriteria(ransac_max_iteration, ransac_max_validation))
  return result


def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range

def standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

def visualize_rre(Trans, rmse, cdists, optim_index):
  rmseList = np.array(rmse)
  cdistsList = np.array(cdists)
  rreList = [np.rad2deg(compute_rre((inv_extrinsic @ t)[:3,:3], gt_T_world[:3,:3])) for t in Trans]
  rteList = [compute_rte((inv_extrinsic @ t)[:3,3], gt_T_world[:3,3]) for t in Trans]
  print("The following gives rre and rte max indices and values:")
  print(np.max(rreList), np.argmax(rreList))
  print(np.min(rteList), np.argmin(rteList))
  # print(rteList[np.argmax(rreList)], np.argmax(rreList))
  rreList, rteList = np.array(rreList), np.array(rteList) 
  # print((rreList>=175))
  # print((rteList<=0.01))
  # np.intersect1d(np.where(rreList>=175), np.where(rteList<=0.01)) 
  goodList = rreList[np.intersect1d(np.where(rreList>=175), np.where(rteList<=0.01))]
  goodListTwo = rreList[np.intersect1d(np.where(rreList<=5), np.where(rteList<=0.01))]
  print("Good list under current parameter sets (one and two):")
  print(goodList, len(goodList))
  print(goodListTwo, len(goodListTwo))
  print("Good list rmse and cdist (one and two):")
  print(rmseList[np.intersect1d(np.where(rreList>=175), np.where(rteList<=0.01))])
  print(cdistsList[np.intersect1d(np.where(rreList>=175), np.where(rteList<=0.01))])
  print(rmseList[np.intersect1d(np.where(rreList<=5), np.where(rteList<=0.01))])
  print(cdistsList[np.intersect1d(np.where(rreList<=5), np.where(rteList<=0.01))])
  print("Fractioned optimal rmse and cdist:")
  print(rmseList[optim_index])
  print(cdistsList[optim_index])
  print("Global optimal rmse and cdist, with indices:")
  print(np.min(rmseList), np.argmin(rmseList))
  print(np.min(cdistsList), np.argmin(cdistsList))
  return 0

def get_o3d_icp_with_global_registration(source_pcd, target_pcd, 
                                         voxel_size=icp_global_voxel_size, 
                                         init_random_time=init_random_time, 
                                         cdistFrac = cdistFrac):
  source_pcd_o3d, target_pcd_o3d = o3d.geometry.PointCloud(), o3d.geometry.PointCloud()
  source_pcd_o3d.points = o3d.utility.Vector3dVector(source_pcd)
  target_pcd_o3d.points = o3d.utility.Vector3dVector(target_pcd)
  source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, source_pcd_o3d, target_pcd_o3d)
  print(source, target)
  print(source_down, target_down)

  # perform multiple RANSAC inits, and choose the best amongest them
  Trans, rmse, cdists = [], [], []
  for i in tqdm(range(init_random_time)):
    result_ransac = execute_global_registration(source_down, target_down, \
                      source_fpfh, target_fpfh, \
                      voxel_size)
    # result_ransac = execute_fast_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
    # print("result_ransac: ")
    # print(result_ransac)
    # print(result_ransac.transformation.shape)
    # o3dvis([getTransPcd(source_down.points, result_ransac.transformation), target_down.points])
    # result_icp = refine_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size, result_ransac.transformation)
    # TODO: Fetch the best initialization as the result
    result_icp = get_o3d_icp(source_down.points, target_down.points, init=result_ransac.transformation)
    # print("result_icp:")
    # print(result_icp)
    tr = result_icp.transformation
    trans_source_down = copy.deepcopy(source_down)
    trans_source_down = trans_source_down.transform(tr)
    dists = target_down.compute_point_cloud_distance(trans_source_down)
    dists = np.asarray(dists)
    # print("chamfer dist:")
    # print(dists)
    # print(dists)
    Trans.append(tr)
    rmse.append(result_icp.inlier_rmse)
    cdists.append(np.sum(dists))
    # draw_registration_result(source_down, target_down, result_icp.transformation)
  print("cdist list:")
  print(cdists)
  print(np.min(cdists))
  print(np.argmin(cdists))
  o3dvis([getTransPcd(source_down.points, Trans[np.argmin(np.array(cdists)+np.array(rmse))]), target_down.points])
  # return Trans[np.argmin(rmse)]
  cdistsNorm = standardization(np.array(cdists))
  rmseNorm = standardization(np.array(rmse))
  optim_index = np.argmin((1-cdistFrac)*rmseNorm+cdistFrac*cdistsNorm)
  # visualize_rre(Trans, rmse, cdists, optim_index)
  visualize_rre(Trans, rmseNorm, cdistsNorm, optim_index)
  return Trans[np.argmin((1-cdistFrac)*rmseNorm+cdistFrac*cdistsNorm)]

def pts_to_o3d_pcd(pts):
  """Transform to o3d pcd"""
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(pts)
  return pcd

def o3d_Vec_to_o3d_pcd(pts):
  """Transform to o3d pcd"""
  pcd = o3d.geometry.PointCloud()
  pcd.points = pts
  return pcd

def o3dvis(pts):
  o3d.visualization.draw_geometries([pts_to_o3d_pcd(pt) for pt in pts])
  return 0

def getTransPcd(source_pcd, T):
  return source_pcd @ T[:3, :3].T + T[:3, 3]

def get_transform(rotation, translation):
    transformation = np.eye(4)
    transformation[:3, :3] = rotation
    transformation[:3, 3] = translation
    return transformation



# Load scene meta info
meta_vis = get_meta(meta_file)
scales = meta_vis["scales"]
print(meta_vis.keys())
print(meta_vis["object_ids"])
print(meta_vis["object_names"])
intrinsic = meta_vis["intrinsic"]
extrinsic = meta_vis["extrinsic"]
inv_extrinsic = np.linalg.inv(extrinsic)
gt_T_world = meta_vis["poses_world"][vis_id]
gt_T = extrinsic @ gt_T_world
# print(gt_T.shape)
# gt_T = meta_vis["poses_world"][vis_id]
# Fetch source point cloud from model dictionary
# print(object_models)


# Reconstruct source and target point clouds (in camera frame)
print(object_name)
object_name = object_models_name[vis_id]
vis_pcd_source = object_models[object_name]
vis_pcd_source = vis_pcd_source * scales[vis_id]
vis_pcd_target = get_object_point_cloud(get_label(label_file), \
                    vis_id, \
                    get_depth(depth_file), \
                    intrinsic)

# o3dvis([vis_pcd_source])

print("src and tg len:")
print(len(vis_pcd_target), len(vis_pcd_source))


# Perform ICP
# source_pcd, target_pcd = vis_pcd_target, vis_pcd_source
source_pcd, target_pcd = vis_pcd_source, vis_pcd_target
# source_pcd = np.hstack((source_pcd, np.ones((len(source_pcd), 1))))
# target_pcd = np.hstack((target_pcd, np.ones((len(target_pcd), 1))))
# source_pcd, target_pcd = (source_pcd @ np.linalg.inv(extrinsic).T), (target_pcd @ np.linalg.inv(extrinsic).T)
# source_pcd = np.array([x[:-1]/x[-1] for x in source_pcd])
# target_pcd = np.array([x[:-1]/x[-1] for x in target_pcd])
# print(source_pcd.shape, target_pcd.shape)
# o3dvis([source_pcd, target_pcd])
T = get_o3d_icp_with_global_registration(source_pcd, target_pcd)
# T = get_o3d_icp_tensor(source_pcd, target_pcd)
# T = get_o3d_icp(source_pcd, target_pcd) # should use T.transformation to get transformation matrix!
# T = icp(source_pcd, target_pcd, 100)
T_world = inv_extrinsic @ T
TrEstiPcd = source_pcd @ T[:3, :3].T + T[:3, 3]
TrGtPcd = source_pcd @ gt_T[:3, :3].T + gt_T[:3, 3]
# TrGtPcd = source_pcd @ gt_T[:3, :3].T + gt_T[:3, 3]


o3dvis([TrEstiPcd, target_pcd])

# o3dvis([TrGtPcd, target_pcd])


print(len(TrEstiPcd), len(TrGtPcd))
# print(TrEstiPcd)
# print(TrGtPcd)
# o3d.visualization.draw_geometries([pts_to_o3d_pcd(TrEstiPcd), pts_to_o3d_pcd(TrGtPcd)])

# T = np.eye(4)
print(source_pcd.shape)
print(target_pcd.shape)
print("GT T:")
print(gt_T)
print("GT T world:")
print(gt_T_world)
print("pred T:")
print(T)
print("pred T world:")
print(T_world)
# rre = np.rad2deg(compute_rre(T[:3, :3], gt_T[:3, :3]))
# rte = compute_rte(T[:3, 3], gt_T[:3, 3])
rre = np.rad2deg(compute_rre(T_world[:3, :3], gt_T_world[:3, :3]))
rte = compute_rte(T_world[:3, 3], gt_T_world[:3, 3])
print(f"rre={rre}, rte={rte}")
# show_points(target_pcd, "Target pcd")
# show_points(source_pcd, "Source pcd")
# show_points(TrEstiPcd, "Transformed source pcd")
# show_points(TrGtPcd, "GT Transformed source pcd")






dict_keys(['poses_world', 'extents', 'scales', 'object_ids', 'object_names', 'extrinsic', 'intrinsic'])
[35 39 48 51 58]
['jenga', 'master_chef_can', 'potted_meat_can', 'pudding_box', 'wood_block']
wooden_puzzle3
src and tg len:
6117 49152
:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.030.
:: Estimate normal with search radius 0.060.
:: Compute FPFH feature with search radius 0.150.
:: Estimate normal with search radius 0.060.
:: Compute FPFH feature with search radius 0.150.
PointCloud with 49152 points. PointCloud with 6117 points.
PointCloud with 4416 points. PointCloud with 6117 points.


100%|██████████| 100/100 [00:18<00:00,  5.42it/s]


cdist list:
[40.838890354689205, 28.13779620600238, 28.1410672672902, 40.52657242806276, 49.075959354355504, 49.17863542012413, 28.11946394776579, 53.99750767596552, 28.13560607965636, 48.186485626604636, 28.144658589455467, 49.64867463202808, 63.40137178316486, 40.59856568572702, 160.90379682172673, 148.70079902114287, 49.466064553216, 40.259667553730225, 48.6633517931776, 47.49001225887349, 48.745680107671895, 48.668086819895414, 48.65037252036005, 40.34916896948722, 39.98957468733967, 49.16036105598759, 40.619438996469995, 28.147027263738327, 131.93002722276526, 48.16546161509025, 28.068803025148128, 134.42394053734805, 49.11858488047333, 28.095218643013652, 62.512014640103715, 28.1390633534278, 47.936921031314284, 92.93833393835583, 39.49803528021863, 130.74728837042534, 48.04588928502494, 39.4223303015782, 28.155369204200394, 49.178879000813836, 48.022630126342094, 28.112953929549704, 28.1073455857653, 49.507629014310524, 40.57035201932845, 48.47363946166535, 48.08039104840375, 28

In [5]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

source_points = vis_pcd_target # RGBD pcd, 6117
target_points = vis_pcd_source # Model pcd, 49000
# print(source_points.shape, target_points.shape)
target_points_o3d = pts_to_o3d_pcd(target_points)

target_points_o3d = target_points_o3d.farthest_point_down_sample(len(source_points)) # heavily down-sampled side
target_points_pts = np.asarray(target_points_o3d.points) 

def transformation_generate(rotation, translation):
    '''
    input:
        rotation 3*3
        translation 3*1
    output:
        transformation 4*4
    '''
    transformation = np.eye(4)
    
    transformation[:3, :3] = rotation
    transformation[:3, 3] = translation
    
    return transformation

def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nxm array of points
        dst: Nxm array of points
    Output:
        distances: Euclidean distances of the nearest neighbor
        indices: dst indices of the nearest neighbor
    '''
    # print(src.shape,dst.shape)
    assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()

def umeyama_alignment(x, y, with_scale = True):
    """
    input:
        x: 3*n
        y: 3*n
        with_scale: calculate scale or not
    output:
        r: rotation
        t: translation
        c: scale
    """

    if x.shape != y.shape:
        raise GeometryException("data matrices must have the same shape")

    # m = dimension, n = nr. of data points
    m, n = x.shape

    # means, eq. 34 and 35
    mean_x = x.mean(axis=1)
    mean_y = y.mean(axis=1)

    # variance, eq. 36
    # "transpose" for column subtraction
    sigma_x = 1.0 / n * (np.linalg.norm(x - mean_x[:, np.newaxis])**2)

    # covariance matrix, eq. 38
    outer_sum = np.zeros((m, m))
    for i in range(n):
        outer_sum += np.outer((y[:, i] - mean_y), (x[:, i] - mean_x))
    cov_xy = np.multiply(1.0 / n, outer_sum)

    # SVD (text betw. eq. 38 and 39)
    u, d, v = np.linalg.svd(cov_xy)  

    # S matrix, eq. 43
    s = np.eye(m)
    if np.linalg.det(u) * np.linalg.det(v) < 0.0:
        # Ensure a RHS coordinate system (Kabsch algorithm).
        s[m - 1, m - 1] = -1

    # rotation, eq. 40
    r = u.dot(s).dot(v)

    # scale & translation, eq. 42 and 41
    c = 1 / sigma_x * np.trace(np.diag(d).dot(s)) if with_scale else 1.0

    t = mean_y - np.multiply(c, r.dot(mean_x))

    return r, t, c

def icp(source, target, max_iterations=100, tolerance=1e-6, init_scale = 1, init_transformation = None):
    """
    iterative get rigid transformation
    input:
        source: N*3
        target: N*3
        init_transformation = 4*4
    output:
        src: transformed point clouds
    """
    src = np.copy(source.T) # 3*N
    dst = np.copy(target.T) # 3*N
    
    if init_transformation is not None:
        r = init_transformation[:3, :3]
        t = init_transformation[:3, 3][:, np.newaxis]
        src = np.multiply(init_scale, np.dot(r, src)) + t
    prev_error = 0
    
    for i in range(max_iterations):
        distances, indices = nearest_neighbor(src.T,dst.T)
        rotation, translation, scale = umeyama_alignment(src, dst[:,indices], with_scale=False)
        # TODO:write into homogeneous coordinates
        src = np.multiply(scale, rotation.dot(src)) + translation[:,np.newaxis]
        mean_error = np.mean(distances)
        
        if np.abs(prev_error-mean_error) < tolerance:
            # last iterative calculate the scale
            rotation, translation, scale = umeyama_alignment(src, dst[:,indices], with_scale=True)
            src = np.multiply(scale, rotation.dot(src)) + translation[:,np.newaxis]
            break
        prev_error = mean_error
    # calc source point clouds Transformation
    rotation, translation, scale = umeyama_alignment(source.T, src, with_scale=True)
    return rotation, translation, scale, src


"""
source_points: from RGBD
target_points: from canonical frame
"""

target_points_pts, source_points = source_points, target_points_pts
# init_trans: TODO can be improved to get better result
t_wlh = np.max(target_points_pts,axis=0)-np.min(target_points_pts,axis=0)
# t_wlh = np.max(target_points,axis=0)-np.min(target_points,axis=0)
s_wlh = np.max(source_points,axis=0)-np.min(source_points,axis=0)
init_scale = np.mean(t_wlh/s_wlh)

# iterative closest points: get rigid transformation result
rotation, translation, scale, Tsrc = icp(
    source_points, target_points_pts, init_scale = init_scale, init_transformation=np.eye(4)
    # source_points, target_points, init_scale = init_scale, init_transformation=np.eye(4)
)

In [6]:
rotation, translation
tr = transformation_generate(rotation, translation)
tr_world = inv_extrinsic @ tr
rre = np.rad2deg(compute_rre(tr_world[:3, :3], gt_T_world[:3, :3]))
rte = compute_rte(tr_world[:3, 3], gt_T_world[:3, 3])
print(f"rre={rre}, rte={rte}")

rre=174.56518010558622, rte=0.016216558186695342


In [7]:
"""Visualize given pcd"""
train_vis_varianct, train_vis_index, object_ids, vis_id = "1-1-1", 0, [35, 39, 48, 51, 58], 58
object_name = object_models_name[vis_id]
meta_vis = get_meta(meta_file)
scales = meta_vis["scales"]
vis_pcd_source = object_models[object_name]
vis_pcd_source = vis_pcd_source * scales[vis_id]
o3d.visualization.draw_geometries([pts_to_o3d_pcd(vis_pcd_source)])




In [8]:
# PointNetDense network (saved)
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
import numpy as np
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import math
import shutil


def rot_6d(x, y):
    # Gram-schmidt process to get the rotation matrix
    x = F.normalize(x, dim=-1)
    y = y - x * (x * y).sum(-1, keepdims=True)
    y = F.normalize(y, dim=-1)
    z = torch.cross(x, y, -1)
    return torch.stack([x, y, z], dim=-1)

class STNkd(nn.Module):
    def __init__(self, k_input=32, k_output=32*32, n_points=256, spread_points=False):
        self.n_points = n_points
        self.spread_points = spread_points
        self.k_input = k_input
        self.k_output = k_output

        super(STNkd, self).__init__()
        self.conv1 = torch.nn.Conv1d(k_input, 64, 1) # feature extractors
        self.conv2 = torch.nn.Conv1d(64, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 512, 1)
        self.bnConv1 = nn.BatchNorm1d(64)
        self.bnConv2 = nn.BatchNorm1d(256)
        self.bnConv3 = nn.BatchNorm1d(512)
        self.relu = nn.ReLU()

        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        if self.spread_points:
          self.fc4 = nn.Linear(128, k_output*n_points)
        else:
          self.fc4 = nn.Linear(128, k_output)
        self.bnDense2 = nn.BatchNorm1d(256)
        self.bnDense3 = nn.BatchNorm1d(128)

        # weight initializations
        for net in [
            self.conv1,
            self.conv2,
            self.conv3,
            self.fc2,
            self.fc3,
            self.fc4
        ]:
            torch.nn.init.xavier_uniform_(net.weight)

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bnConv1(self.conv1(x)))
        x = F.relu(self.bnConv2(self.conv2(x)))
        x = F.relu(self.bnConv3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 512)

        x = F.relu(self.bnDense2(self.fc2(x)))
        x = F.relu(self.bnDense3(self.fc3(x)))
        x = self.fc4(x)

        if self.spread_points:
          x = x.view(-1, self.k_output, self.n_points)
        return x

class PointNet(nn.Module):
    def __init__(self, global_feature_size=81, n_points=256): # global feature = object id (size=79) + scene id (size=2)
        super(PointNet, self).__init__()
        self.n_points = n_points
        self.global_feature_size = global_feature_size
        self.stn3d_input_feat = STNkd(k_input=3, k_output=3*3, n_points=self.n_points, spread_points=True)
        self.conv1 = torch.nn.Conv1d(3*3, 64, 1) # local feature extraction layers
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 256, 1)
        self.conv4 = torch.nn.Conv1d(256, 512, 1)
        self.conv5 = torch.nn.Conv1d(512, 1024, 1)
        self.bnConv1 = nn.BatchNorm1d(64)
        self.bnConv2 = nn.BatchNorm1d(128)
        self.bnConv3 = nn.BatchNorm1d(256)
        self.bnConv4 = nn.BatchNorm1d(512)
        self.bnConv5 = nn.BatchNorm1d(1024)

        self.branch1 = torch.nn.Linear(global_feature_size, 256) # global feature extraction layers
        self.bnBranch1 = nn.BatchNorm1d(256)
        self.branch2 = torch.nn.Linear(256, 1024) # global feature extraction layers
        self.bnBranch2 = nn.BatchNorm1d(1024)
        self.dense2_r = torch.nn.Linear(2048, 1024) # inference layers
        self.dense2_t = torch.nn.Linear(2048, 1024)
        self.bnDense2_r = nn.BatchNorm1d(1024)
        self.bnDense2_t = nn.BatchNorm1d(1024)
        self.dense3_r = torch.nn.Linear(1024, 512)
        self.dense3_t = torch.nn.Linear(1024, 512)
        self.bnDense3_r = nn.BatchNorm1d(512)
        self.bnDense3_t = nn.BatchNorm1d(512)
        self.dense4_r = torch.nn.Linear(512, 256)
        self.dense4_t = torch.nn.Linear(512, 256)
        self.bnDense4_r = nn.BatchNorm1d(256)
        self.bnDense4_t = nn.BatchNorm1d(256)
        self.dense5_r = torch.nn.Linear(256, 128)
        self.dense5_t = torch.nn.Linear(256, 128)
        self.bnDense5_r = nn.BatchNorm1d(128)
        self.bnDense5_t = nn.BatchNorm1d(128)
        self.dense6_r = torch.nn.Linear(128, 6) # output layer for rotation
        self.dense6_t = torch.nn.Linear(128, 3) # output layer for translation

        # weight initializations
        for net in [
            # self.conv0,
            self.conv1,
            self.conv2,
            self.conv3,
            self.conv4,
            self.conv5,
            self.branch1,
            self.branch2,
            self.dense2_r,
            self.dense2_t,
            self.dense3_r,
            self.dense3_t,
            self.dense4_r,
            self.dense4_t,
            self.dense5_r,
            self.dense5_t,
            self.dense6_r,
            self.dense6_t
        ]:
            torch.nn.init.xavier_uniform_(net.weight)

    def forward(self, x, label, scene):
        points = x[:, :3]  # batch_size, 3, n_points
        # colors = x[:, 3:]

        # normalize to [-1,1] centered at (0,0,0)
        mins = points.min(dim=2, keepdim=True).values
        maxs = points.max(dim=2, keepdim=True).values
        center = (mins + maxs) / 2
        half_extents = (maxs - mins) / 2
        longest = half_extents.max(dim=1, keepdim=True).values.clamp(
            min=1e-3
        )
        points = (points - center) / longest

        # pcd feature extractions
        x = points

        # initial transform with spatial transformer
        x = self.stn3d_input_feat(x)
        x = F.relu(self.bnConv1(self.conv1(x)))
        x = F.relu(self.bnConv2(self.conv2(x)))
        x = F.relu(self.bnConv3(self.conv3(x)))
        x = F.relu(self.bnConv4(self.conv4(x)))
        x = F.relu(self.bnConv5(self.conv5(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        # global feature extractions
        globalFeature = torch.cat((label, scene), dim=1)
        globalFeature = self.bnBranch1(self.branch1(globalFeature))
        globalFeature = self.bnBranch2(self.branch2(globalFeature))

        # concatenate local and global features, and transform using a transformer
        x = torch.cat((x, globalFeature), dim=1)
        x_rot = x
        x_tran = x_rot.clone()

        # infer rotation
        x_rot = F.relu(self.bnDense2_r(self.dense2_r(x_rot)))
        x_rot = F.relu(self.bnDense3_r(self.dense3_r(x_rot)))
        x_rot = F.relu(self.bnDense4_r(self.dense4_r(x_rot)))
        x_rot = F.relu(self.bnDense5_r(self.dense5_r(x_rot)))
        x_rot = self.dense6_r(x_rot)
        x_rot = rot_6d(x_rot[..., 0:3], x_rot[..., 3:6]) # fetch output rotation matrix

        x_tran = F.relu(self.bnDense2_t(self.dense2_t(x_tran)))
        x_tran = F.relu(self.bnDense3_t(self.dense3_t(x_tran)))
        x_tran = F.relu(self.bnDense4_t(self.dense4_t(x_tran)))
        x_tran = F.relu(self.bnDense5_t(self.dense5_t(x_tran)))
        x_tran = self.dense6_t(x_tran)
        x_tran = x_tran * longest.view(-1, 1) + center.view_as(x_tran) # scale back and un-center (batch_size, 3), to get translation vector
        return x_tran, x_rot


OSError: [WinError 126] 找不到指定的模块。 Error loading "c:\Users\NamShoo\miniconda3\lib\site-packages\torch\lib\caffe2_nvrtc.dll" or one of its dependencies.

In [ ]:
# Start ICP estimation for all test data
!mkdir test_outputs_pointnet_icp_1
import numpy as np
from tqdm import tqdm
import os
import copy

# Flag for continuing from previous steps
ctu = False
ctu_variant, ctu_index = 0, 0
if ctu:
  ctu_variant, ctu_index = "2-50-1", 149

# Output resulting dictionary
dataset_type = "test"
output_dir = "./{}_outputs_pointnet_icp_1".format(dataset_type)
test_size = 200

# pointnet initializer loading
init_dict = load_json("/content/result_test_pointnet.json")

# For each object at each testing data: Get scene object's point cloud (as target pcd)
result_dict = {}
if ctu:
  json_path = os.path.join(output_dir, "result_till_{}_index_{}.json".format(ctu_variant, ctu_index))
  result_dict = load_json(json_path)
print(test_depth_files)
print(test_meta_files)
print(test_label_files)
print(test_prefix_ids)

# for test_depth_path, test_meta_path, test_label_path, test_prefix_id in tqdm(zip(test_depth_files, test_meta_files, test_label_files, test_prefix_ids)):
for i in range(ctu_index, test_size):
  if (i % 10 == 0):
    print("Auto-uploading test outputs to cloud at epoch {}".format(i))
    !cp -r /content/test_outputs_pointnet_icp_1 /content/drive/MyDrive/
  test_depth_path, test_meta_path, test_label_path, test_prefix_id = test_depth_files[i], test_meta_files[i], test_label_files[i], test_prefix_ids[i]
  print("Test data id {}".format(test_prefix_id))
  test_meta, test_depth, test_seg_label = get_meta(test_meta_path), get_depth(test_depth_path), get_label(test_label_path)   # convert from mm to m
  intrinsic = test_meta["intrinsic"]
  extrinsic = test_meta["extrinsic"]
  inv_extrinsic = np.linalg.inv(extrinsic)
  scales = test_meta["scales"]
  result_dict[test_prefix_id] = {}
  result_dict[test_prefix_id]["poses_world"] = [None] * object_models_num
  for object_id, object_name in zip(test_meta["object_ids"], test_meta["object_names"]):
    # print(test_meta["object_ids"], test_meta["object_names"])
    print("Object id {}, name {}".format(object_id, object_name))
    # if (object_id != 56):
      # continue
    # mask the object's depth map to get test_object_depth
    # print(object_id, type(object_id))
    # print(np.sum(test_seg_label==object_id))
    if (np.sum(test_seg_label==object_id) == 0):
      print("Object not found at scene.")
      continue
    test_image_label = test_seg_label.copy()

    # mask the object's depth map to get test_object_depth
    # print(test_image_label[test_image_label==30])
    # print(test_image_label[test_image_label==51])
    test_pcd_target = get_object_point_cloud(test_image_label, object_id, test_depth, intrinsic)

    # Fetch source point cloud from model dictionary
    test_pcd_source = object_models[object_name]
    test_pcd_source *= scales[object_id] # scale the model pcd using factors
    if (not test_pcd_source.any()):
      print("Model pcd are all origins. Weird...")
      continue
    # print(test_pcd_source.shape)
    # print(test_pcd_target.shape)

    # ICP to poinget posture
    init = init_dict[test_prefix_id]["poses_world"][object_id]
    init_cam = extrinsic @ init # map to camera frame

    T = get_o3d_icp_with_pointnet_init(test_pcd_source, test_pcd_target, init=init_cam)
    T_world = inv_extrinsic @ T # Transform to world coordinate
    # TODO: may simply take PointNet's rotation and ICP's translation as results
    # T_world[:3, :3] = init[:3, :3]

    # update resulting dictionary with object id and pose list
    result_dict[test_prefix_id]["poses_world"][object_id] = T_world.tolist()
  if (((i+1)%10)==0):
    output_name = "result_till_{}_index_{}.json".format(test_prefix_id, i)
    dump_json(result_dict, os.path.join(output_dir, output_name))

print("Test result:")
print(result_dict)
print(len(result_dict))



# Output resulting dictionary
output_name = "result_pointnet_icp_1.json"
dump_json(result_dict, os.path.join(output_dir, output_name))
!cp /content/test_outputs_pointnet_icp_1/result_pointnet_icp_1.json /content/drive/MyDrive/outputs
!cp -r /content/test_outputs_pointnet_icp_1 /content/drive/MyDrive/


